In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)

#from pandas import DataFrame, read_csv
import pandas as pd

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

import sys  
#sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
from AdaBandFlt import *

%matplotlib tk

In [3]:
#load data
# file path of csv file
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'
Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [4]:
#load data
# file path of csv file
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2_tstamp.csv'
Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2_tstamp.csv'

# create dataframe
df_ = pd.read_csv(Location, sep=',')

In [5]:
df.head()

,El 21,El 31,El 41,El 22,El 32,El 42,El 23,El 33,El 43,El 15
%t,,,,,,,,,,
0.00,-2.61,1.02,0.80,1.93,-1.25,-1.59,-0.57,4.09,-1.14,-0.34
0.04,-3.30,0.00,2.05,1.25,-1.02,-0.45,-1.82,2.61,-0.45,-0.80
0.08,-2.73,0.11,2.73,1.48,0.45,1.14,-1.14,3.52,-0.23,-0.80
0.12,-3.07,-0.45,1.25,0.80,-0.57,-0.57,-1.25,2.50,-1.02,-0.57
0.16,-2.39,1.48,0.34,1.14,0.00,-0.57,-0.34,1.48,-0.23,0.45


In [6]:
df_.head()

,T[s],sppa0001_21_Timestamp_Unit0,T[s].1,sppa0001_31_Timestamp_Unit0,T[s].2,sppa0001_41_Timestamp_Unit0,T[s].3,sppa0001_22_Timestamp_Unit0,T[s].4,sppa0001_32_Timestamp_Unit0,T[s].5,sppa0001_42_Timestamp_Unit0,T[s].6,sppa0001_23_Timestamp_Unit0,T[s].7,sppa0001_33_Timestamp_Unit0,T[s].8,sppa0001_43_Timestamp_Unit0,T[s].9,sppa0001_15_Timestamp_Unit0
0,1.63776,NaN,0.38532,NaN,0.85184,NaN,2.14396,NaN,2.13620,NaN,0.56064,NaN,2.14936,NaN,0.47808,NaN,2.13872,NaN,7.96416,NaN
1,1.99384,NaN,1.08808,NaN,2.12424,NaN,2.15492,NaN,2.14376,NaN,2.14380,NaN,2.15504,NaN,0.90124,NaN,2.14168,NaN,16.50048,NaN
2,2.13476,NaN,2.11608,NaN,2.13104,NaN,2.16148,NaN,2.15196,NaN,2.15204,NaN,2.16224,NaN,2.11860,NaN,2.14432,NaN,17.73908,NaN
3,2.13824,NaN,2.12060,NaN,2.13352,NaN,2.16732,NaN,2.15752,NaN,2.15772,NaN,2.17080,NaN,2.14616,NaN,2.15276,NaN,NaN,NaN
4,2.14420,NaN,2.13140,NaN,2.13628,NaN,2.17520,NaN,2.16072,NaN,2.16072,NaN,2.17296,NaN,2.14848,NaN,2.15540,NaN,NaN,NaN


In [5]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

#xminnoise = int(np.round(11114*(fs/1000)))
#xmaxnoise = int(np.round(18511*(fs/1000)))

#noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

#xminspike = int(np.round(130826*(fs/1000)))
#xmaxspike = int(np.round(131699*(fs/1000)))

#burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

#begin_data = signal_filtered.iloc[:500000]

C:\Users\SYL21\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [6]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [7]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

"""plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')"""

"plt.figure()\nplt.plot(noise_levels)\nplt.grid(True)\nplt.xlabel('Time')\nplt.ylabel('Noise Amplitude [µV]')\nplt.title('Noise Levels')"

In [31]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0012, 
                           noise_window_size = 0.01,
                           threshold_factor = 3.5,
                           maxseparation = 0.001)  
                
#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,indice_depass_positif,indice_depass_negatif,indice_1er_depass,indice_zero_central,i_max-i_min,Delta_amplitudes
0,53278,53286,53277,53285,53277,53281,-8,16.237153
1,53391,53375,53389,53373,53373,53386,16,18.401586
2,53492,53504,53489,53499,53489,53497,-12,31.926962
3,53645,53652,53643,53652,53643,53649,-7,17.140363
4,53922,53939,53910,53931,53910,53927,-17,44.081780
5,53960,53952,53959,53948,53948,53957,8,28.400890
6,54115,54133,54106,54126,54106,54123,-18,61.367703
7,54261,54248,54256,54245,54245,54253,13,27.291692
8,54314,54329,54302,54322,54302,54320,-15,59.721773
9,54481,54491,54464,54488,54464,54486,-10,49.451979


In [32]:
signal = signal_filtered

In [28]:
spike_data = record_spikes(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.001,
                  t_after = 0.002)

(273, 76)


In [11]:
toto = df_.iloc[:, 2].dropna().values
arr = toto*25000
spike_info_ = pd.DataFrame(arr.astype('int') , columns = ['indice_1er_depass'])


In [12]:
spike_data_ = record_spikes(signal, fs, spike_info_,
                  'indice_1er_depass',
                  t_before = 0.002,
                  t_after = 0.002)



(466, 101)


In [37]:
print_spikes(spike_data,
             t_before_alignement = 0.001,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

In [40]:
print_spikes(spike_data_,
             t_before_alignement = 0.002,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

In [27]:
signal = full_signal

In [30]:
spike_data = record_spikes(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.001,
                  t_after = 0.002)

spike_data_ = record_spikes(signal, fs, spike_info_,
                  'indice_1er_depass',
                  t_before = 0.001,
                  t_after = 0.002)

(273, 76)
(466, 76)


In [31]:
print_spikes(spike_data,
             t_before_alignement = 0.001,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

In [32]:
print_spikes(spike_data_,
             t_before_alignement = 0.001,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

In [21]:
spike_data_ = record_spikes_oneline(signal, fs, spike_info_,
                  'indice_1er_depass',
                  t_before = 0.002,
                  t_after = 0.002)

In [23]:
plt.figure()
plt.plot(signal)
plt.plot(spike_data_)
plt.grid(True)

In [33]:
spike_data = record_spikes_oneline(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.002,
                  t_after = 0.002)

In [34]:
plt.figure()
plt.plot(signal)
plt.plot(spike_data)
plt.grid(True)

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\SYL21\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 227, in resize
    self.draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 12, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 437, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1493, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\image.py", line 141, in _draw_list_compositing_image

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\SYL21\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "C:\Users\SYL21\Anaconda3\lib\tkinter\__init__.py", line 746, in callit
    func(*args)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 310, in idle_draw
    self.draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 12, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 437, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1493, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\Users

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\SYL21\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "C:\Users\SYL21\Anaconda3\lib\tkinter\__init__.py", line 746, in callit
    func(*args)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 310, in idle_draw
    self.draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 12, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 437, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\SYL21\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1493, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\Users

In [45]:
sophie_spike = spike_info_['indice_1er_depass'].values
our_spike = spike_info['indice_1er_depass'].values
temporal_tol = 0.0005

i = 0
j = 0

joint_spikes_sophie_side = []
solo_spikes_sophie_side = []
joint_spikes_our_side = []
solo_spikes_our_side = []
tol = int(np.round(fs * temporal_tol))

while(True):
    if sophie_spike[i] < our_spike[j] - tol:
        solo_spikes_sophie_side.append(i)
        i += 1
    elif sophie_spike[i] > our_spike[j] + tol:
        solo_spikes_our_side.append(j)
        j += 1
    else:
        joint_spikes_sophie_side.append(i)
        joint_spikes_our_side.append(j)
        i += 1
        j += 1
    if(i == len(sophie_spike)-1 or j == len(our_spike)-1):
        break
    
print(len(sophie_spike))
print(len(our_spike))
print('joint_spikes_sophie_side')
print(len(joint_spikes_sophie_side))
print(joint_spikes_sophie_side)
print('solo_spikes_sophie_side')
print(len(solo_spikes_sophie_side))
print(solo_spikes_sophie_side)
print('joint_spikes_our_side')
print(len(joint_spikes_our_side))
print(joint_spikes_our_side)
print('solo_spikes_our_side')
print(len(solo_spikes_our_side))
print(solo_spikes_our_side)

465
376
joint_spikes_sophie_side
125
[4, 10, 13, 16, 18, 22, 23, 25, 29, 38, 45, 49, 51, 53, 58, 59, 73, 77, 78, 84, 91, 93, 94, 97, 98, 102, 103, 110, 112, 113, 114, 122, 123, 125, 126, 127, 128, 129, 132, 135, 138, 141, 146, 153, 155, 159, 165, 168, 170, 179, 184, 186, 188, 190, 193, 196, 199, 202, 208, 211, 212, 214, 215, 218, 219, 225, 228, 245, 247, 251, 252, 253, 268, 279, 282, 286, 291, 295, 300, 301, 303, 307, 309, 311, 319, 323, 325, 329, 331, 336, 338, 341, 348, 356, 364, 371, 374, 375, 380, 382, 393, 396, 397, 399, 400, 403, 405, 409, 414, 416, 418, 419, 422, 423, 426, 427, 429, 431, 438, 439, 442, 446, 447, 456, 458]
solo_spikes_sophie_side
335
[0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 14, 15, 17, 19, 20, 21, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 46, 47, 48, 50, 52, 54, 55, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 92, 95, 96, 99, 100, 101, 104, 105, 106, 107, 108, 109, 111

In [56]:
spike_data = record_spikes(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.002,
                  t_after = 0.002)

spike_data_ = record_spikes(signal, fs, spike_info_,
                  'indice_1er_depass',
                  t_before = 0.002,
                  t_after = 0.002)

(377, 101)
(466, 101)


In [59]:
print_spikes(spike_data.iloc[:,joint_spikes_our_side],
             t_before_alignement = 0.001,
             randomize = True,
             nb_spike = 20,
             fs = 25000)

In [67]:
spike_data.iloc[:,joint_spikes_our_side]

,0,4,6,8,9,11,12,16,21,29,...,347,351,353,359,360,363,364,365,370,371
0,0.0,5.610770,9.126536,29.918936,-3.659566,0.255873,16.845110,5.105538,-15.567847,-3.452557,...,-5.189212,14.399266,-14.641793,-0.445477,-7.712627,-4.634376,3.116054,-0.162473,-1.490249,-4.873439
1,1.0,2.134023,5.188855,25.973381,-0.487254,-0.610777,18.269571,3.793541,-15.464376,-3.213822,...,-2.964910,11.951948,-14.575016,-2.494156,-7.329479,-3.036356,3.366723,0.247655,-2.612888,-5.012035
2,2.0,-1.606497,1.402590,21.773397,2.511800,-2.366074,19.613241,2.948831,-14.346428,-3.121515,...,-0.421410,9.344806,-12.966388,-3.869768,-7.225383,-1.961479,3.591971,0.639986,-3.036948,-5.366745
3,3.0,-5.220426,-1.754744,17.931439,5.300395,-5.004469,20.938516,2.428843,-12.034583,-3.420687,...,2.126042,6.751025,-10.305252,-4.245155,-7.546957,-1.585335,3.769620,0.865282,-2.993422,-5.681416
4,4.0,-8.351226,-3.993425,14.837945,7.861902,-8.290001,22.234611,2.015745,-8.465291,-4.306114,...,4.345749,4.339007,-7.197130,-3.471106,-8.287595,-1.979867,3.865374,0.802993,-2.706744,-5.682939
5,5.0,-10.715618,-5.200115,12.620987,10.137233,-11.841965,23.391557,1.510769,-3.772833,-5.845565,...,5.995828,2.276727,-4.211333,-1.623499,-9.312317,-3.094146,3.823485,0.413480,-2.313852,-5.160449
6,6.0,-12.124028,-5.386756,11.186942,12.006574,-15.259912,24.207820,0.825928,1.653382,-7.930504,...,7.000995,0.726043,-1.768020,1.004509,-10.414430,-4.751735,3.574455,-0.232790,-1.839705,-4.022878
7,7.0,-12.490681,-4.634817,10.314958,13.324098,-18.237161,24.435855,0.039309,7.170517,-10.273543,...,7.468189,-0.181015,-0.094639,3.969450,-11.379525,-6.666793,3.060443,-0.961775,-1.229764,-2.323995
8,8.0,-11.843776,-3.063608,9.763891,13.987457,-20.623127,23.855218,-0.609372,11.961404,-12.460407,...,7.634079,-0.379946,0.746894,6.787943,-12.035273,-8.479272,2.269578,-1.539209,-0.417527,-0.254044
9,9.0,-10.339706,-0.831332,9.353927,14.005356,-22.420438,22.350953,-0.787250,15.183295,-14.047721,...,7.764926,0.090416,0.787598,9.058081,-12.276183,-9.806996,1.263283,-1.739810,0.606642,1.896417


In [63]:
print_spikes(spike_data.iloc[:,solo_spikes_our_side],
             t_before_alignement = 0.001,
             randomize = True,
             nb_spike = 20,
             fs = 25000)

In [64]:
print_spikes(spike_data_.iloc[:,joint_spikes_sophie_side],
             t_before_alignement = 0.001,
             randomize = True,
             nb_spike = 20,
             fs = 25000)

In [62]:
print_spikes(spike_data_.iloc[:,solo_spikes_sophie_side],
             t_before_alignement = 0.001,
             randomize = True,
             nb_spike = 20,
             fs = 25000)